In [1]:
case class Passenger(name: String, cellPhoneNumber: Option[String])
case class Carriage(passengers: List[Passenger])
case class Train(name: String, carriages: List[Carriage])
case class Route(name: String, activeTrain: Option[Train])

defined class Passenger
defined class Carriage
defined class Train
defined class Route

In [2]:
val route1 = Route("Glen Gach to Glen Pach",
    Some(Train("The Flying Scotsman",
      List(Carriage(List(Passenger("Rob Roy", Some("121-212-1212")), Passenger("Connor McCleod", None))),
           Carriage(List(Passenger("Joey McDougall", Some("454-545-4545")))))
    ))
)

val route2 = Route("Defuncto 1", None)

val route3 = Route("Busy Route of Luddites",
    Some(Train("The Tech Express",
      List(Carriage(List(Passenger("Ug", None), Passenger("Glug", None))),
           Carriage(Nil),
           Carriage(List(Passenger("Smug", Some("323-232-3232")))))
    ))
)

val routes = List(route1, route2, route3)

route1: Route = Route(Glen Gach to Glen Pach,Some(Train(The Flying Scotsman,List(Carriage(List(Passenger(Rob Roy,Some(121-212-1212)), Passenger(Connor McCleod,None))), Carriage(List(Passenger(Joey McDougall,Some(454-545-4545))))))))
route2: Route = Route(Defuncto 1,None)
route3: Route = Route(Busy Route of Luddites,Some(Train(The Tech Express,List(Carriage(List(Passenger(Ug,None), Passenger(Glug,None))), Carriage(List()), Carriage(List(Passenger(Smug,Some(323-232-3232))))))))
routes: List[Route] = List(
  Route(Glen Gach to Glen Pach,Some(Train(The Flying Scotsman,List(Carriage(List(Passenger(Rob Roy,Some(121-212-1212)), Passenger(Connor McCleod,None))), Carriage(List(Passenger(Joey McDougall,Some(454-545-4545)))))))),
  Route(Defuncto 1,None),
  Route(Busy Route of Luddites,Some(Train(The Tech Express,List(Carriage(List(Passenger(Ug,None), Passenger(Glug,None))), Carriage(List()), Carriage(List(Passenger(Smug,Some(323-232-3232))))))))
)

In [2]:
for {
    route <- routes
    active <- route.activeTrain   // huh!
    carriage <- active.carriages
    passenger <- carriage.passengers
    number <- passenger.cellPhoneNumber
} yield number

: 

In [2]:
routes.flatMap { route =>  // Seq  (flatMap A => Seq[B])
    route.activeTrain.flatMap { active =>  // Option  (flatMap A => Option[B]) // these two
        active.carriages.flatMap { carriage =>  // Seq  (flatMap A => Seq[B])  // are the problem...
            carriage.passengers.flatMap { passenger =>  // Seq
                passenger.cellPhoneNumber.map { number =>  // Option
                    number
                }
            }
        }
    }
}

: 

In [3]:
for {
    route <- routes
    active <- route.activeTrain.toSeq   // recommended whenever mixing options and seqs
    carriage <- active.carriages
    passenger <- carriage.passengers
    number <- passenger.cellPhoneNumber.toSeq  // unnecessary here, but still clear
} yield number

res2: List[String] = List("121-212-1212", "454-545-4545", "323-232-3232")

In [4]:
import scala.concurrent._
import ExecutionContext.Implicits.global
import duration._


import scala.concurrent._
import ExecutionContext.Implicits.global
import duration._

In [4]:
val fListONums = Future(List(1,2,3,4,5))
def square(x: Int): Future[Int] = Future(x * x)

for {
    nums <- fListONums
    num <- nums    // doh! - no mixie!
    sq <- square(num)
} yield sq

: 

In [5]:
val fListONums = Future(List(1,2,3,4,5))
def square(x: Int): Future[Int] = Future(x * x)

for {
    nums <- fListONums
    squares <- Future.traverse(nums)(x => square(x))  // Seq[Int] & Int => Future[Int] => Future[Seq[Int]]
} yield squares

fListONums: Future[List[Int]] = Success(List(1, 2, 3, 4, 5))
defined function square
res4_2: Future[List[Int]] = Success(List(1, 4, 9, 16, 25))